In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
.appName("assignment 3")\
.getOrCreate()

spark

**Task Set – Intermediate to Advanced PySpark (No DLT)**

**Data Ingestion & Schema Handling**

In [2]:
# 1. Load the CSV using inferred schema.
from google.colab import drive
drive.mount('/content/drive')

df_emp_ts = spark.read.csv('/content/drive/MyDrive/employee_timesheet.csv',header= True,inferSchema=True)
df_emp_ts.show()

Mounted at /content/drive
+----------+-----+----------+-------+---------+----------+---------+------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|
+----------+-----+----------+-------+---------+----------+---------+------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|
+----------+-----+----------+-------+---------+----------+---------+------+



In [3]:
# 2. Load the same file with schema explicitly defined.
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType

schema = StructType([
    StructField("EmployeeID", StringType(), True),
    StructField("Name", StringType(), True),
    StructField("Department", StringType(), True),
    StructField("Project", StringType(), True),
    StructField("WorkHours", IntegerType(), True),
    StructField("WorkDate", DateType(), True),
    StructField("Location", StringType(), True),
    StructField("Mode", StringType(), True),
])

df_emp_ts = spark.read.option("header", True).schema(schema).csv("/content/drive/MyDrive/employee_timesheet.csv")
df_emp_ts.show()
df_emp_ts.printSchema()

+----------+-----+----------+-------+---------+----------+---------+------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|
+----------+-----+----------+-------+---------+----------+---------+------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|
+----------+-----+----------+-------+---------+----------+---------+------+

root
 |-- EmployeeID: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- Project: string (nullable = true)
 |-- WorkHours: integer (nullable = true)
 |-- WorkDate: date (nullable = t

In [4]:
# 3. Add a new column Weekday extracted from WorkDate
from pyspark.sql.functions import date_format

df_with_weekday = df_emp_ts.withColumn("Weekday", date_format("WorkDate", "EEEE"))
df_with_weekday.show()

+----------+-----+----------+-------+---------+----------+---------+------+---------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|  Weekday|
+----------+-----+----------+-------+---------+----------+---------+------+---------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|Wednesday|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|Wednesday|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote| Thursday|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|   Friday|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|   Friday|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote| Saturday|
+----------+-----+----------+-------+---------+----------+---------+------+---------+



**Aggregations & Grouping**

In [5]:
# 4. Calculate total work hours by employee.
from pyspark.sql.functions import sum

df_total_hours = df_emp_ts.groupBy("EmployeeID", "Name").agg(sum("WorkHours").alias("TotalHours"))
df_total_hours.show()

+----------+-----+----------+
|EmployeeID| Name|TotalHours|
+----------+-----+----------+
|      E103| John|         5|
|      E104|Meena|         6|
|      E102|  Raj|        15|
|      E101|Anita|        17|
+----------+-----+----------+



In [6]:
# 5. Calculate average work hours per department.
from pyspark.sql.functions import avg

df_avg_dept = df_emp_ts.groupBy("Department").agg(avg("WorkHours").alias("AvgHours"))
df_avg_dept.show()

+----------+-----------------+
|Department|         AvgHours|
+----------+-----------------+
|        HR|              7.5|
|   Finance|              5.0|
|        IT|7.666666666666667|
+----------+-----------------+



In [7]:
# 6. Get top 2 employees by total hours using window function.
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window_spec = Window.orderBy(df_total_hours["TotalHours"].desc())

df_top_2 = df_total_hours.withColumn("rank", row_number().over(window_spec)).filter("rank <= 2")
df_top_2.show()

+----------+-----+----------+----+
|EmployeeID| Name|TotalHours|rank|
+----------+-----+----------+----+
|      E101|Anita|        17|   1|
|      E102|  Raj|        15|   2|
+----------+-----+----------+----+



**Date Operations**

In [8]:
# 7. Filter entries where WorkDate falls on a weekend.
from pyspark.sql.functions import dayofweek

df_weekends = df_emp_ts.filter(dayofweek("WorkDate").isin([1, 7]))
df_weekends.show()

+----------+----+----------+-------+---------+----------+--------+------+
|EmployeeID|Name|Department|Project|WorkHours|  WorkDate|Location|  Mode|
+----------+----+----------+-------+---------+----------+--------+------+
|      E102| Raj|        HR|   Beta|        8|2024-05-04|  Mumbai|Remote|
+----------+----+----------+-------+---------+----------+--------+------+



In [9]:
# 8. Calculate running total of hours per employee using window.
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

window_spec_emp = Window.partitionBy("EmployeeID").orderBy("WorkDate").rowsBetween(Window.unboundedPreceding, Window.currentRow)

df_running_total = df_emp_ts.withColumn("RunningTotalHours", sum("WorkHours").over(window_spec_emp))
df_running_total.show()

+----------+-----+----------+-------+---------+----------+---------+------+-----------------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|RunningTotalHours|
+----------+-----+----------+-------+---------+----------+---------+------+-----------------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|                8|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|               17|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|                7|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|               15|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|                5|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|                6|
+----------+-----+----------+-------+---------+----------+---------+------+-----------------+



**Joining DataFrames**


In [10]:
# 9. Create department_location.csv :
# Department,DeptHead
# IT,Anand
# HR,Shruti
# Finance,Kamal

df_dep_ts = spark.read.csv('/content/drive/MyDrive/department_location.csv',header= True,inferSchema=True)
df_dep_ts.show()

+----------+--------+
|Department|DeptHead|
+----------+--------+
|        IT|   Anand|
|        HR|  Shruti|
|   Finance|   Kamal|
+----------+--------+



In [11]:
# 10. Join with timesheet data and list all employees with their DeptHead.
df_joined = df_emp_ts.join(df_dep_ts, on="Department", how="left")
df_joined.show()

+----------+----------+-----+-------+---------+----------+---------+------+--------+
|Department|EmployeeID| Name|Project|WorkHours|  WorkDate| Location|  Mode|DeptHead|
+----------+----------+-----+-------+---------+----------+---------+------+--------+
|        IT|      E101|Anita|  Alpha|        8|2024-05-01|Bangalore|Remote|   Anand|
|        HR|      E102|  Raj|   Beta|        7|2024-05-01|   Mumbai|Onsite|  Shruti|
|   Finance|      E103| John|  Alpha|        5|2024-05-02|    Delhi|Remote|   Kamal|
|        IT|      E101|Anita|  Alpha|        9|2024-05-03|Bangalore|Remote|   Anand|
|        IT|      E104|Meena|  Gamma|        6|2024-05-03|Hyderabad|Onsite|   Anand|
|        HR|      E102|  Raj|   Beta|        8|2024-05-04|   Mumbai|Remote|  Shruti|
+----------+----------+-----+-------+---------+----------+---------+------+--------+



**Pivot & Unpivot**

In [12]:
# 11. Pivot table: total hours per employee per project.

df_pivot = df_emp_ts.groupBy("EmployeeID", "Name").pivot("Project").sum("WorkHours")
df_pivot.show()

+----------+-----+-----+----+-----+
|EmployeeID| Name|Alpha|Beta|Gamma|
+----------+-----+-----+----+-----+
|      E103| John|    5|NULL| NULL|
|      E104|Meena| NULL|NULL|    6|
|      E102|  Raj| NULL|  15| NULL|
|      E101|Anita|   17|NULL| NULL|
+----------+-----+-----+----+-----+



In [14]:
# 12. Unpivot example: Convert mode-specific hours into rows.
from pyspark.sql.functions import when,expr

df_mode_hours = df_emp_ts.withColumn("RemoteHours", when(df_emp_ts["Mode"] == "Remote", df_emp_ts["WorkHours"]).otherwise(0))\
.withColumn("OnsiteHours", when(df_emp_ts["Mode"] == "Onsite", df_emp_ts["WorkHours"]).otherwise(0))

df_unpivoted = df_mode_hours.select("EmployeeID", "Name", "WorkDate",
    expr("stack(2, 'Remote', RemoteHours, 'Onsite', OnsiteHours) as (ModeType, Hours)")
)
df_unpivoted.show()

+----------+-----+----------+--------+-----+
|EmployeeID| Name|  WorkDate|ModeType|Hours|
+----------+-----+----------+--------+-----+
|      E101|Anita|2024-05-01|  Remote|    8|
|      E101|Anita|2024-05-01|  Onsite|    0|
|      E102|  Raj|2024-05-01|  Remote|    0|
|      E102|  Raj|2024-05-01|  Onsite|    7|
|      E103| John|2024-05-02|  Remote|    5|
|      E103| John|2024-05-02|  Onsite|    0|
|      E101|Anita|2024-05-03|  Remote|    9|
|      E101|Anita|2024-05-03|  Onsite|    0|
|      E104|Meena|2024-05-03|  Remote|    0|
|      E104|Meena|2024-05-03|  Onsite|    6|
|      E102|  Raj|2024-05-04|  Remote|    8|
|      E102|  Raj|2024-05-04|  Onsite|    0|
+----------+-----+----------+--------+-----+



**UDF & Conditional Logic**


In [15]:
# 13. Create a UDF to classify work hours:
# def workload_tag(hours):
# if hours >= 8: return "Full"
# elif hours >= 4: return "Partial"
# else: return "Light"

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def workload_tag(hours):
    if hours >= 8:
        return "Full"
    elif hours >= 4:
        return "Partial"
    else:
        return "Light"

workload_udf = udf(workload_tag, StringType())

In [16]:
# 14. Add a column WorkloadCategory using this UDF.

df_with_category = df_emp_ts.withColumn("WorkloadCategory", workload_udf("WorkHours"))
df_with_category.show()

+----------+-----+----------+-------+---------+----------+---------+------+----------------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|WorkloadCategory|
+----------+-----+----------+-------+---------+----------+---------+------+----------------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|            Full|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|         Partial|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|         Partial|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|            Full|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|         Partial|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|            Full|
+----------+-----+----------+-------+---------+----------+---------+------+----------------+



**Nulls and Cleanup**


In [18]:
# 15. Introduce some nulls in Mode column.
from pyspark.sql.functions import when

df_with_nulls = df_emp_ts.withColumn("Mode",when(df_emp_ts["EmployeeID"] == "E102", None).otherwise(df_emp_ts["Mode"]))
df_with_nulls.show()

+----------+-----+----------+-------+---------+----------+---------+------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|
+----------+-----+----------+-------+---------+----------+---------+------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|  NULL|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|  NULL|
+----------+-----+----------+-------+---------+----------+---------+------+



In [19]:
# 16. Fill nulls with "Not Provided".

df_filled = df_with_nulls.fillna({"Mode": "Not Provided"})
df_filled.show()

+----------+-----+----------+-------+---------+----------+---------+------------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|        Mode|
+----------+-----+----------+-------+---------+----------+---------+------------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|      Remote|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Not Provided|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|      Remote|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|      Remote|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|      Onsite|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Not Provided|
+----------+-----+----------+-------+---------+----------+---------+------------+



In [20]:
# 17. Drop rows where WorkHours < 4.

df_cleaned = df_filled.filter(df_filled["WorkHours"] >= 4)
df_cleaned.show()

+----------+-----+----------+-------+---------+----------+---------+------------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|        Mode|
+----------+-----+----------+-------+---------+----------+---------+------------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|      Remote|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Not Provided|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|      Remote|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|      Remote|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|      Onsite|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Not Provided|
+----------+-----+----------+-------+---------+----------+---------+------------+



**Advanced Conditions**

In [21]:
# 18. Use when-otherwise to mark employees as "Remote Worker" if >80% entries are Remote.

from pyspark.sql.functions import col, count, when

df_remote_stats = df_emp_ts.groupBy("EmployeeID").agg(count("*").alias("TotalEntries"),count(when(col("Mode") == "Remote", True)).alias("RemoteEntries"))

from pyspark.sql.functions import expr, when

df_remote_flagged = df_remote_stats.withColumn("RemoteRatio", col("RemoteEntries") / col("TotalEntries")).withColumn(
    "RemoteTag", when(col("RemoteRatio") > 0.8, "Remote Worker").otherwise("Hybrid/Onsite"))

df_remote_flagged.show()

+----------+------------+-------------+-----------+-------------+
|EmployeeID|TotalEntries|RemoteEntries|RemoteRatio|    RemoteTag|
+----------+------------+-------------+-----------+-------------+
|      E103|           1|            1|        1.0|Remote Worker|
|      E104|           1|            0|        0.0|Hybrid/Onsite|
|      E101|           2|            2|        1.0|Remote Worker|
|      E102|           2|            1|        0.5|Hybrid/Onsite|
+----------+------------+-------------+-----------+-------------+



In [22]:
# 19. Add a new column ExtraHours where hours > 8.

df_with_extra = df_emp_ts.withColumn("ExtraHours", when(col("WorkHours") > 8, col("WorkHours") - 8).otherwise(0))
df_with_extra.show()

+----------+-----+----------+-------+---------+----------+---------+------+----------+
|EmployeeID| Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|ExtraHours|
+----------+-----+----------+-------+---------+----------+---------+------+----------+
|      E101|Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|         0|
|      E102|  Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|         0|
|      E103| John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|         0|
|      E101|Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|         1|
|      E104|Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|         0|
|      E102|  Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|         0|
+----------+-----+----------+-------+---------+----------+---------+------+----------+



**Union + Duplicate Handling**


In [23]:
# 20. Append a dummy timesheet for new interns using unionByName() .

from pyspark.sql import Row

intern_data = [
    Row(EmployeeID="E201", Name="Ishaan", Department="IT", Project="Gamma", WorkHours=6, WorkDate="2024-05-05", Location="Chennai", Mode="Remote"),
    Row(EmployeeID="E202", Name="Nitya", Department="HR", Project="Beta", WorkHours=5, WorkDate="2024-05-05", Location="Pune", Mode="Onsite")
]

df_interns = spark.createDataFrame(intern_data).withColumn("WorkDate", col("WorkDate").cast("date"))

df_combined = df_emp_ts.unionByName(df_interns)
df_combined.show()

+----------+------+----------+-------+---------+----------+---------+------+
|EmployeeID|  Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|
+----------+------+----------+-------+---------+----------+---------+------+
|      E101| Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|
|      E102|   Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|
|      E103|  John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|
|      E101| Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|
|      E104| Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|
|      E102|   Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|
|      E201|Ishaan|        IT|  Gamma|        6|2024-05-05|  Chennai|Remote|
|      E202| Nitya|        HR|   Beta|        5|2024-05-05|     Pune|Onsite|
+----------+------+----------+-------+---------+----------+---------+------+



In [24]:
# 21. Remove duplicate rows based on all columns.

df_deduplicated = df_combined.dropDuplicates()
df_deduplicated.show()

+----------+------+----------+-------+---------+----------+---------+------+
|EmployeeID|  Name|Department|Project|WorkHours|  WorkDate| Location|  Mode|
+----------+------+----------+-------+---------+----------+---------+------+
|      E104| Meena|        IT|  Gamma|        6|2024-05-03|Hyderabad|Onsite|
|      E101| Anita|        IT|  Alpha|        9|2024-05-03|Bangalore|Remote|
|      E103|  John|   Finance|  Alpha|        5|2024-05-02|    Delhi|Remote|
|      E101| Anita|        IT|  Alpha|        8|2024-05-01|Bangalore|Remote|
|      E102|   Raj|        HR|   Beta|        8|2024-05-04|   Mumbai|Remote|
|      E102|   Raj|        HR|   Beta|        7|2024-05-01|   Mumbai|Onsite|
|      E201|Ishaan|        IT|  Gamma|        6|2024-05-05|  Chennai|Remote|
|      E202| Nitya|        HR|   Beta|        5|2024-05-05|     Pune|Onsite|
+----------+------+----------+-------+---------+----------+---------+------+

